## Imports

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components
from datetime import date, timedelta, datetime

import os

## Load dataframe from newest file

In [2]:
for file in os.listdir('newest'):
    if "csv" in file:
        print("Loading "+file+" ...")

df_mx = pd.read_csv('./newest/'+file, encoding = "ISO-8859-1")
print(file+" Loaded")

Loading 201014COVID19MEXICO.csv ...
201014COVID19MEXICO.csv Loaded


## Active Cases of Mexico

In [39]:
df = df_mx

df = df.loc[(df['RESULTADO_LAB'] == 1)] # resultado = positivo SARS-COV-2 o Pendiente

# df = df = df.loc[df['ENTIDAD_RES'] == 2] #entidad = Baja California

# df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali

df = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']

df_by_date = df.groupby("FECHA_SINTOMAS",as_index=False).count()
df_by_date['FECHA_SINTOMAS'] = pd.to_datetime(df_by_date['FECHA_SINTOMAS'])

first_case_date = df_by_date['FECHA_SINTOMAS'].iloc[0]

last_case_date = df_by_date['FECHA_SINTOMAS'].iloc[-1]

delta = (last_case_date) - first_case_date       # as timedelta

active_df = pd.DataFrame()

for i in range(delta.days + 1):
    day = first_case_date + timedelta(days=i)

    df_by_date['FECHA'] = day
    
    active_date_minus_14_days = day - timedelta(days=11)
    mask = (df_by_date['FECHA_SINTOMAS'] >= active_date_minus_14_days) & (df_by_date['FECHA_SINTOMAS'] <= day)
    filtered_df=df_by_date.loc[mask]
    
    filtered_df = filtered_df.groupby(["FECHA"],as_index=False).sum()
    active_df = active_df.append(filtered_df)

    
fig = px.line(active_df, active_df.FECHA, active_df.CONFIRMADO,
              title='Activos Mexico por Estado')
fig.show()

## Actives Cases by State

In [40]:
df = df_mx

di = {1:"Aguascalientes", 2: "Baja California", 3:"Baja California Sur",  4: "Campeche", 5: "Coahuila", 6: "Colima",
     7: "Chiapas", 8: "Chihuahua", 9: "CDMX", 10: "Durango", 11: "Guanajuato", 12: "Guerrero", 13: "Hidalgo",
     14: "Jalisco", 15: "Mexico", 16: "Michoacan", 17: "Morelos", 18: "Nayarit", 19: "Nuevo Leon", 20: "Oaxaca",
     21: "Puebla", 22: "Queretaro", 23: "Quintana Roo", 24: "San Luis Potosi", 25: "Sinaloa", 26: "Sonora",
     27: "Tabasco", 28: "Tamaulipas", 29: "Tlaxcala", 30: "Veracruz", 31: "Yucatan", 32: "Zacatecas"}

df = df.loc[(df['RESULTADO_LAB'] == 1)] # resultado = positivo SARS-COV-2 o Pendiente

df = df.replace({"ENTIDAD_RES": di})

df = df[['FECHA_SINTOMAS','RESULTADO_LAB','ENTIDAD_RES']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']

df_by_date = df.groupby(["FECHA_SINTOMAS","ENTIDAD_RES"],as_index=False).count()
df_by_date['FECHA_SINTOMAS'] = pd.to_datetime(df_by_date['FECHA_SINTOMAS'])

first_case_date = df_by_date['FECHA_SINTOMAS'].iloc[0]

last_case_date = df_by_date['FECHA_SINTOMAS'].iloc[-1]

delta = (last_case_date) - first_case_date       # as timedelta

active_df = pd.DataFrame()

for i in range(delta.days + 1):
    day = first_case_date + timedelta(days=i)

    df_by_date['FECHA'] = day
    
    active_date_minus_14_days = day - timedelta(days=11)
    mask = (df_by_date['FECHA_SINTOMAS'] >= active_date_minus_14_days) & (df_by_date['FECHA_SINTOMAS'] <= day)
    filtered_df=df_by_date.loc[mask]
    
    filtered_df = filtered_df.groupby(["FECHA","ENTIDAD_RES"],as_index=False).sum()
    active_df = active_df.append(filtered_df)

fig = px.line(active_df, active_df.FECHA, active_df.CONFIRMADO, line_group= "ENTIDAD_RES",
              color="ENTIDAD_RES",
              title='Activos Mexico por Estado')
fig.show()

## Active Cases of Baja California

In [18]:
df = df_mx

di = {1:"Ensenada", 2: "Mexicali", 3:"Tecate",  4: "Tijuana", 5: "Rosarito"}

df = df.loc[(df['RESULTADO_LAB'] == 1)] # resultado = positivo SARS-COV-2 o Pendiente

df = df = df.loc[df['ENTIDAD_RES'] == 2] #entidad = Baja California

df = df.replace({"MUNICIPIO_RES": di})

df = df[['FECHA_SINTOMAS','RESULTADO_LAB','MUNICIPIO_RES']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']

df_by_date = df.groupby(["FECHA_SINTOMAS","MUNICIPIO_RES"],as_index=False).count()
df_by_date['FECHA_SINTOMAS'] = pd.to_datetime(df_by_date['FECHA_SINTOMAS'])

first_case_date = df_by_date['FECHA_SINTOMAS'].iloc[0]

last_case_date = df_by_date['FECHA_SINTOMAS'].iloc[-1]

delta = (last_case_date) - first_case_date       # as timedelta

active_df = pd.DataFrame()

for i in range(delta.days + 1):
    day = first_case_date + timedelta(days=i)

    df_by_date['FECHA'] = day
    
    active_date_minus_14_days = day - timedelta(days=11)
    mask = (df_by_date['FECHA_SINTOMAS'] >= active_date_minus_14_days) & (df_by_date['FECHA_SINTOMAS'] <= day)
    filtered_df=df_by_date.loc[mask]
    
    filtered_df = filtered_df.groupby(["FECHA","MUNICIPIO_RES"],as_index=False).sum()
    active_df = active_df.append(filtered_df)

fig = px.line(active_df, active_df.FECHA, active_df.CONFIRMADO, line_group= "MUNICIPIO_RES",
              color="MUNICIPIO_RES",
              title='Activos Baja California por Municipio')
fig.show()

## Active Cases of Baja California considering population by County 

In [41]:
df = df_mx

di = {1:"Ensenada", 2: "Mexicali", 3:"Tecate",  4: "Tijuana", 5: "Rosarito"}

population = {1: 486639, 2: 988417, 3: 102406, 4: 1641570, 5: 96734}


df = df.loc[(df['RESULTADO_LAB'] == 1)] # resultado = positivo SARS-COV-2 o Pendiente

df = df.loc[df['ENTIDAD_RES'] == 2] #entidad = Baja California

df = df[['FECHA_SINTOMAS','RESULTADO_LAB','MUNICIPIO_RES']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']

df_by_date = df.groupby(["FECHA_SINTOMAS","MUNICIPIO_RES"],as_index=False).count()
df_by_date['FECHA_SINTOMAS'] = pd.to_datetime(df_by_date['FECHA_SINTOMAS'])

first_case_date = df_by_date['FECHA_SINTOMAS'].iloc[0]

last_case_date = df_by_date['FECHA_SINTOMAS'].iloc[-1]

delta = (last_case_date) - first_case_date       # as timedelta

active_df = pd.DataFrame()

for i in range(delta.days + 1):
    day = first_case_date + timedelta(days=i)

    df_by_date['FECHA'] = day
    
    active_date_minus_14_days = day - timedelta(days=11)
    mask = (df_by_date['FECHA_SINTOMAS'] >= active_date_minus_14_days) & (df_by_date['FECHA_SINTOMAS'] <= day)
    filtered_df=df_by_date.loc[mask]
    
    filtered_df = filtered_df.groupby(["FECHA","MUNICIPIO_RES"],as_index=False).sum()
    active_df = active_df.append(filtered_df)

active_df['POPULATION'] = active_df['MUNICIPIO_RES'].map(population)
active_df['CONFIRMADO'] = (active_df['CONFIRMADO']/active_df['POPULATION'])*10000
    
active_df = active_df.replace({"MUNICIPIO_RES": di})
fig = px.line(active_df, active_df.FECHA, active_df.CONFIRMADO, line_group= "MUNICIPIO_RES",
              color="MUNICIPIO_RES",
              title='Activos Baja California por Municipio por cada 10,000 habitantes')
fig.show()

## Active Cases of Mexicali

In [17]:
df = df_mx

df = df.loc[(df['RESULTADO_LAB'] == 1)] # resultado = positivo SARS-COV-2 o Pendiente

df = df = df.loc[df['ENTIDAD_RES'] == 2] #entidad = Baja California

df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali

df = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']

df_by_date = df.groupby("FECHA_SINTOMAS",as_index=False).count()
df_by_date['FECHA_SINTOMAS'] = pd.to_datetime(df_by_date['FECHA_SINTOMAS'])

first_case_date = df_by_date['FECHA_SINTOMAS'].iloc[0]

last_case_date = df_by_date['FECHA_SINTOMAS'].iloc[-1]

delta = (last_case_date) - first_case_date       # as timedelta

active_df = pd.DataFrame()

for i in range(delta.days + 1):
    day = first_case_date + timedelta(days=i)

    df_by_date['FECHA'] = day
    
    active_date_minus_14_days = day - timedelta(days=11)
    mask = (df_by_date['FECHA_SINTOMAS'] >= active_date_minus_14_days) & (df_by_date['FECHA_SINTOMAS'] <= day)
    filtered_df=df_by_date.loc[mask]
    
    filtered_df = filtered_df.groupby(["FECHA"],as_index=False).sum()
    active_df = active_df.append(filtered_df)

fig = px.line(active_df, active_df.FECHA, active_df.CONFIRMADO,
              title='Activos Mexicali')
fig.show()